In [1]:
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Activation
import numpy as np
import pandas as pd
from keras import initializers

import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import load_model
from keras import optimizers
from keras.callbacks import EarlyStopping
from tqdm import tqdm

Using TensorFlow backend.


# architecture

In [2]:
# construct the model used for classifier
# In our case the input shape is (42,1) is the board flattened

def classifier():  
    
    model = Sequential()
    # 2 "convo+pooling" layers
    model.add(Conv2D(filters=5, kernel_size=(3,3), strides=1, input_shape=(6,7,1)))
    #model.add(MaxPooling2D(pool_size=(2, 2),strides=2))
    model.add(Activation('relu'))
    
    model.add(Conv2D(filters=10, kernel_size=(3,3), strides=1))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=2))
    model.add(Activation('relu'))

    #flatten & linear layers
    model.add(Flatten())
    #model.add(Dense(42))
    model.add(Activation('relu'))

    #Classify if there is a merger waveform, output probabilistic
    model.add(Dense(2, activation="softmax"))

    return model

# compile

In [22]:
# Build model
model = classifier()

# compile model
ad = optimizers.Adam(lr=0.00005)
model.compile(loss='mean_squared_error', optimizer=ad, metrics=['accuracy'])

# functions for training data

In [4]:
# return a 6*7 array of zeros, which represent 
def newgame():
    n = np.zeros(42)
    n = np.reshape(n, (6,7))
    return n

In [5]:
# Check if col th column of B is full

def colAvailable(B, col):
    # start with checking if bottom (row 6) of ith column is empty 
    row = 5
    while B[row, col] != 0:
        row -= 1
        if row < 0:
            return False #column not available cuz full
    return True

In [6]:
# assume col not a full column, meaning it has been run through the columnfull function
# play a piece in ith column, [0,6] of B
# player denoted with -1, 1

# The function avoids modifying the original object numpy array B, returns a new object

def move(B, col, player):
    A = np.array(B)
    # start with checking if bottom (row 6) of ith column is empty 
    row = 5
    while A[row, col] != 0:
        row -= 1
    A[row, col] = player
    return A

In [7]:
# return all possible next board position, 
# and the associated probability of the current player winning based on the current model of network

def possibilities(B, player, network):
    pos = []
    for col in range(7):
        if colAvailable(B, col):
            moved0 = move(B, col, player)
            moved = moved0.reshape(6,7,1)
            moved = np.asarray([moved])
            pos.append([moved0, model.predict(moved)] )
    return pos

In [8]:
#Written by Ryan

def goodMove(B, network, player):
    poss = possibilities(B, network, player)
    
    if(player == 1):
        myIndex = 0
    else:
        myIndex=1
    
    total = 0
    u = np.random.uniform(0,1)
    
    for p in poss:
        total += p[1][0][myIndex]**3
        
    for i in range(len(poss)):
        sumCubeBefore=0
        for j in range(i):
    
            sumCubeBefore += poss[j][1][0][myIndex]**3
            
            
        #print("SumCubeBefore: " + str(sumCubeBefore))
        #print("u: " + str(u))
        #print("myIndex: " + str(myIndex))
        #print("poss[i][1]" + str(poss[i][1]))
        #print("sumCubeBefore + poss[i][1][myIndex]**3/total: " + str((sumCubeBefore + poss[i][1][0][myIndex]**3))
        #print("poss[i][1][myIndex]**3: " + str(poss[i][1][myIndex]**3))
        #print("poss[i][1]**3: " + str(poss[i][1][myIndex]**3) )
        #print(u>=sumCubeBefore and u<=sumCubeBefore + poss[i][1][0][myIndex]**3) 
            
            
        if u>=sumCubeBefore/total and u<=(sumCubeBefore + poss[i][1][0][myIndex]**3)/total:
            A=poss[i][0]
    '''
    print(str(A))
    '''
    
    return A
    

In [9]:
def check_draw(board):
    check = 0
    for c in range(7-3):
        for r in range(6):
            if board[r][c] == 0:
                check+=1
    
    if check == 0:
        return True
    else:
        return False



In [10]:
# board is the current position, a 2d numpy array 
# checkwin checks if any side wins
def checkwin(board):   #returns 1 if four 1s in a row, -1 if four -1s in a row, 0 otherwise
    #board = board[0]
    #print("This is board after" + str(board))
    
    isWin = 0

    wincombopos = [1,1,1,1]
    wincomboneg = [-1,-1,-1,-1]

    for row in board: #look for 4 of a kind on rows
        for i in range(4):
            if (np.array_equal(wincombopos, row[i:i+4])):
                isWin = 1
            elif (np.array_equal(wincomboneg, row[i:i+4])):
                isWin = -1
            
    for column in np.transpose(board): #look for 4 of a kind on columns
        for i in range(3):
            if (np.array_equal(wincombopos, column[i:i+4])):
                isWin = 1
            elif (np.array_equal(wincomboneg, column[i:i+4])):
                isWin = -1
    
    for ri in range(3): #row index   #look for 4 of a kind on downward sloping diagonals
        for ci in range(4): #column index
            diag = np.array([board[ri][ci], board[ri+1][ci+1], board[ri+2][ci+2], board[ri+3][ci+3]])
            if (np.array_equal(wincombopos, diag)):
                isWin = 1
            elif (np.array_equal(wincomboneg, diag)):
                isWin = -1
            
    for ri in range(3):
        for ci in range(4):
            diag = np.array([board[ri+3][ci], board[ri+2][ci+1], board[ri+1][ci+2], board[ri][ci+3]])
            if (np.array_equal(wincombopos, diag)):
                isWin = 1
            if (np.array_equal(wincomboneg, diag)):
                isWin = -1

    return isWin
    

In [11]:
# player1 goes first

def game(network, player1):
    B = newgame()
    step = 0
    boards = []
    probs = []
    
    if player1 == 1:
        player2 = -1
    else:
        player2 = 1
    
    # keep playing as long as no one winning
    while checkwin(B) == 0 and check_draw(B) != True:
        step += 1
        '''
        print()
        print("Step ", step)
        '''
        if (step % 2) == 0: # odd step player1 moves, otherwise player2 moves
            B = goodMove(B, player2, network)
        else:
            B = goodMove(B, player1, network)
        B0 = B.reshape(6,7,1)
        boards.append(B0)
    
    # try linear function first
    if checkwin(B) != 0:
        prob = np.arange(0.5, 1, 0.5/step) + 0.5/step
    else:
        prob = np.full(step, .5)
    
    X = boards
    Y = np.concatenate((prob,1-prob)).reshape(step,2)
        
    return X, Y


In [12]:
# number of training data to generate 

def generateData(N, network, player1):
    X = np.asarray([])
    Y = np.asarray([])
    for i in range(N):
        g = game(model, 1)
        if i == 0:
            X = g[0]
            Y = np.asarray(g[1])
        else:
            X= X +g[0]
            Y = np.concatenate((Y, np.asarray(g[1])))     
        
    return np.asarray(X), np.asarray(Y)

# Generate training data

In [13]:
N = 1000
X,Y = generateData(N, model, 1)

In [14]:
print("We use ", len(X), "training data.")

We use  21364 training data.


In [15]:
N = 100
Xtest,Ytest = generateData(N, model, 1)

In [16]:
print("We use ", len(Xtest), "test data.")

We use  2202 test data.


# train model

In [23]:
# Save the pre-trained classifier
model.save('model0.h5')  # creates a HDF5 file 'my_model.h5'

In [24]:
# Pre_train score record
pre_score = model.evaluate(X, Y, verbose=1)
print("Before training:")
print('train loss:', pre_score[0])
print('train accuracy:', pre_score[1])
print()

# Pre_test score record
pre_score = model.evaluate(Xtest, Ytest, verbose=1)
print("Before training")
print('Test loss:', pre_score[0])
print('Test accuracy:', pre_score[1])
print()

21364/21364 [==============================] - 0s 15us/step
Before training:
train loss: 0.09155864342335703
train accuracy: 0.5164295075940095

2202/2202 [==============================] - 0s 15us/step
Before training
Test loss: 0.09216086397947343
Test accuracy: 0.48819255227938657



In [64]:
# fit network
def learnOnData(X, Y, model):
    epoch = 100
    batch_size = 5

    earlystopping_callback = EarlyStopping(monitor='val_acc',verbose=1,min_delta=0.5,patience=30,baseline=None)

    history = model.fit(X, Y, validation_split = 0.20, batch_size=batch_size, epochs=epoch, verbose=1,
                   callbacks=[earlystopping_callback])
    
    return model

# Performance and Save model

In [26]:
score = model.evaluate(Xtest, Ytest, verbose=1)
print("After training:")
print('Test loss:', score[0])
print('Test accuracy:', score[1])

2202/2202 [==============================] - 0s 18us/step
After training:
Test loss: 0.08898158756496688
Test accuracy: 0.7252497728795598


In [77]:
def bigDaddyLearning(model):
    learningCycles = 35
    
    for i in range(learningCycles):
        N = 7000
        X,Y = generateData(N, model, 1)
        model = learnOnData(X, Y, model)
        print("Done with training cycle " + str(i+1))
        
    return model     
        

In [78]:
model1 = load_model("model1.h5")



model2 = bigDaddyLearning(model1)

Train on 119312 samples, validate on 29829 samples
Epoch 1/100
119312/119312 [==============================] - 24s 204us/step - loss: 0.0884 - acc: 0.7305 - val_loss: 0.0883 - val_acc: 0.7633
Epoch 2/100
119312/119312 [==============================] - 24s 204us/step - loss: 0.0883 - acc: 0.7398 - val_loss: 0.0883 - val_acc: 0.7573
Epoch 3/100
119312/119312 [==============================] - 24s 204us/step - loss: 0.0883 - acc: 0.7441 - val_loss: 0.0883 - val_acc: 0.7473
Epoch 4/100
119312/119312 [==============================] - 24s 204us/step - loss: 0.0883 - acc: 0.7464 - val_loss: 0.0883 - val_acc: 0.7468
Epoch 5/100
119312/119312 [==============================] - 24s 204us/step - loss: 0.0883 - acc: 0.7488 - val_loss: 0.0883 - val_acc: 0.7329
Epoch 6/100
119312/119312 [==============================] - 24s 205us/step - loss: 0.0883 - acc: 0.7494 - val_loss: 0.0883 - val_acc: 0.7481
Epoch 7/100
119312/119312 [==============================] - 24s 205us/step - loss: 0.0883 - acc:

119880/119880 [==============================] - 24s 203us/step - loss: 0.0882 - acc: 0.7661 - val_loss: 0.0888 - val_acc: 0.7628
Epoch 27/100
119880/119880 [==============================] - 24s 202us/step - loss: 0.0882 - acc: 0.7648 - val_loss: 0.0888 - val_acc: 0.7627
Epoch 28/100
119880/119880 [==============================] - 24s 201us/step - loss: 0.0882 - acc: 0.7658 - val_loss: 0.0888 - val_acc: 0.7658
Epoch 29/100
119880/119880 [==============================] - 24s 202us/step - loss: 0.0882 - acc: 0.7644 - val_loss: 0.0888 - val_acc: 0.7910
Epoch 30/100
119880/119880 [==============================] - 24s 202us/step - loss: 0.0882 - acc: 0.7663 - val_loss: 0.0888 - val_acc: 0.7729
Epoch 31/100
119880/119880 [==============================] - 24s 203us/step - loss: 0.0882 - acc: 0.7669 - val_loss: 0.0888 - val_acc: 0.7646
Epoch 00031: early stopping
Done with training cycle 2
Train on 119426 samples, validate on 29857 samples
Epoch 1/100
119426/119426 [======================

119742/119742 [==============================] - 24s 200us/step - loss: 0.0880 - acc: 0.7675 - val_loss: 0.0884 - val_acc: 0.7608
Epoch 21/100
119742/119742 [==============================] - 24s 201us/step - loss: 0.0880 - acc: 0.7695 - val_loss: 0.0884 - val_acc: 0.7508
Epoch 22/100
119742/119742 [==============================] - 24s 201us/step - loss: 0.0880 - acc: 0.7681 - val_loss: 0.0884 - val_acc: 0.7643
Epoch 23/100
119742/119742 [==============================] - 24s 202us/step - loss: 0.0880 - acc: 0.7682 - val_loss: 0.0884 - val_acc: 0.7591
Epoch 24/100
119742/119742 [==============================] - 24s 201us/step - loss: 0.0880 - acc: 0.7683 - val_loss: 0.0884 - val_acc: 0.7574
Epoch 25/100
119742/119742 [==============================] - 24s 203us/step - loss: 0.0880 - acc: 0.7691 - val_loss: 0.0884 - val_acc: 0.7531
Epoch 26/100
119742/119742 [==============================] - 24s 201us/step - loss: 0.0880 - acc: 0.7681 - val_loss: 0.0884 - val_acc: 0.7558
Epoch 27/100

119900/119900 [==============================] - 24s 202us/step - loss: 0.0878 - acc: 0.7689 - val_loss: 0.0886 - val_acc: 0.7460
Epoch 15/100
119900/119900 [==============================] - 24s 202us/step - loss: 0.0878 - acc: 0.7680 - val_loss: 0.0886 - val_acc: 0.7613
Epoch 16/100
119900/119900 [==============================] - 24s 202us/step - loss: 0.0878 - acc: 0.7678 - val_loss: 0.0886 - val_acc: 0.7816
Epoch 17/100
119900/119900 [==============================] - 24s 201us/step - loss: 0.0878 - acc: 0.7691 - val_loss: 0.0886 - val_acc: 0.7838
Epoch 18/100
119900/119900 [==============================] - 24s 202us/step - loss: 0.0878 - acc: 0.7690 - val_loss: 0.0886 - val_acc: 0.7719
Epoch 19/100
119900/119900 [==============================] - 24s 203us/step - loss: 0.0878 - acc: 0.7691 - val_loss: 0.0886 - val_acc: 0.7716
Epoch 20/100
119900/119900 [==============================] - 24s 202us/step - loss: 0.0878 - acc: 0.7697 - val_loss: 0.0886 - val_acc: 0.7703
Epoch 21/100

119803/119803 [==============================] - 24s 201us/step - loss: 0.0883 - acc: 0.7706 - val_loss: 0.0882 - val_acc: 0.7742
Epoch 9/100
119803/119803 [==============================] - 24s 201us/step - loss: 0.0883 - acc: 0.7714 - val_loss: 0.0882 - val_acc: 0.7625
Epoch 10/100
119803/119803 [==============================] - 24s 201us/step - loss: 0.0883 - acc: 0.7698 - val_loss: 0.0882 - val_acc: 0.7673
Epoch 11/100
119803/119803 [==============================] - 24s 201us/step - loss: 0.0883 - acc: 0.7703 - val_loss: 0.0882 - val_acc: 0.7871
Epoch 12/100
119803/119803 [==============================] - 24s 201us/step - loss: 0.0883 - acc: 0.7714 - val_loss: 0.0882 - val_acc: 0.7689
Epoch 13/100
119803/119803 [==============================] - 24s 202us/step - loss: 0.0883 - acc: 0.7714 - val_loss: 0.0882 - val_acc: 0.7555
Epoch 14/100
119803/119803 [==============================] - 24s 202us/step - loss: 0.0883 - acc: 0.7709 - val_loss: 0.0882 - val_acc: 0.7668
Epoch 15/100


Epoch 2/100
119720/119720 [==============================] - 24s 198us/step - loss: 0.0882 - acc: 0.7670 - val_loss: 0.0884 - val_acc: 0.7589
Epoch 3/100
119720/119720 [==============================] - 24s 198us/step - loss: 0.0882 - acc: 0.7670 - val_loss: 0.0883 - val_acc: 0.7710
Epoch 4/100
119720/119720 [==============================] - 24s 198us/step - loss: 0.0882 - acc: 0.7685 - val_loss: 0.0884 - val_acc: 0.7585
Epoch 5/100
119720/119720 [==============================] - 24s 198us/step - loss: 0.0882 - acc: 0.7669 - val_loss: 0.0883 - val_acc: 0.7623
Epoch 6/100
119720/119720 [==============================] - 24s 198us/step - loss: 0.0882 - acc: 0.7683 - val_loss: 0.0883 - val_acc: 0.7751
Epoch 7/100
119720/119720 [==============================] - 24s 200us/step - loss: 0.0882 - acc: 0.7692 - val_loss: 0.0884 - val_acc: 0.7583
Epoch 8/100
119720/119720 [==============================] - 24s 198us/step - loss: 0.0882 - acc: 0.7686 - val_loss: 0.0884 - val_acc: 0.7822
Epoch 

119001/119001 [==============================] - 24s 198us/step - loss: 0.0883 - acc: 0.7694 - val_loss: 0.0888 - val_acc: 0.7675
Epoch 28/100
119001/119001 [==============================] - 24s 198us/step - loss: 0.0883 - acc: 0.7697 - val_loss: 0.0888 - val_acc: 0.7658
Epoch 29/100
119001/119001 [==============================] - 24s 198us/step - loss: 0.0883 - acc: 0.7701 - val_loss: 0.0888 - val_acc: 0.7600
Epoch 30/100
119001/119001 [==============================] - 24s 198us/step - loss: 0.0883 - acc: 0.7686 - val_loss: 0.0888 - val_acc: 0.7693
Epoch 31/100
119001/119001 [==============================] - 24s 198us/step - loss: 0.0883 - acc: 0.7701 - val_loss: 0.0888 - val_acc: 0.7625
Epoch 00031: early stopping
Done with training cycle 11
Train on 119688 samples, validate on 29922 samples
Epoch 1/100
119688/119688 [==============================] - 24s 198us/step - loss: 0.0885 - acc: 0.7691 - val_loss: 0.0885 - val_acc: 0.7363
Epoch 2/100
119688/119688 [======================

KeyboardInterrupt: 

In [79]:
# Save the trained classifier
model.save('model3.h5')  # creates a HDF5 file 'my_model.h5'

# Compete between old and new models to test our training

In [38]:
def optMove(B, network, player):
    poss = possibilities(B, network, player)
    
    if(player == 1):
        myIndex = 0
    else:
        myIndex=1
    
    p = poss[0][1][0][myIndex]
    opti = 0
    for i in range(len(poss)):
        pi = poss[i][1][0][myIndex]
        if pi > p:
            opti = i 
    
    optB = poss[opti][0]
                          
    return optB
    

In [56]:
# allow 2 models to play each other
# Network 1 is the player1 "1", "1" goes first
# output who wins "1", "-1", or a draw "0"

def compete(network1, network2):
    B = newgame()
    step = 0
    
    # keep playing as long as no one winning
    while checkwin(B) == 0 and check_draw(B) != True:
        step += 1
        if (step % 2) == 0: # odd step player1 moves, otherwise player2 moves
            B = goodMove(B, -1, network2)
        else:
            B = goodMove(B, 1, network1)
    return checkwin(B)

In [57]:
# return percentage of times player1 win, draw, and player-1 win

def competeMultiple(N, network1, network2):
    draw  = 0
    win1 = 0
    win_1 = 0
    for i in range(N):
        result = compete(network1, network2)
        if result == 0:
            draw += 1
        elif result == 1:
            win1 += 1
        else:
            win_1 += 1
    percent = float(win_1)/N
    
    return percent
    

In [58]:
# Check model performance accuracy before
model0 = load_model("model0.h5")
model1 = load_model("model1.h5")

In [80]:
model3 = load_model("model3.h5")

In [61]:
competeMultiple(10000, model0, model1)

0.4405

In [62]:
competeMultiple(10000, model1, model1)

0.4416

In [82]:
competeMultiple(1000, model0, model3)

0.431

In [83]:
competeMultiple(1000, model3, model0)

0.434

### competeMultiple(1000, model0, model2)

In [ ]:
model2.save('model2.h5')